In [1]:
!pip install -U altair

# Clustering Crypto

In [2]:
# Initial imports
import pandas as pd
from pathlib import Path
from IPython.display import display
import altair as alt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Fetching Cryptocurrency Data

In [3]:
# Alternatively, use the provided csv file:
file_path = Path("../Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [4]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[
    ['CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']
]
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [5]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.drop(
    crypto_df[crypto_df['IsTrading'] == False].index
)


In [6]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[
    crypto_df['Algorithm'] != 'NaN'
]


In [7]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(
   columns=['IsTrading']
)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [8]:
# Count null values
crypto_df.isnull().sum()


CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64

In [9]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [10]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.drop(
    crypto_df[
        crypto_df['TotalCoinsMined'] == 0
    ].index
)
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [11]:
# Drop rows where there are 'N/A' text values
for column in crypto_df.columns:
    crypto_df = crypto_df.drop(crypto_df[crypto_df[column] == 'N/A'].index)
crypto_df.head()


/Users/soma/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [12]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df['CoinName'], index=crypto_df.index)
coins_name.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [13]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(
    columns=['CoinName']
)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [14]:
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].astype(float)
crypto_df.dtypes

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply    float64
dtype: object

In [15]:
# Create dummy variables for text features
features_enc = pd.get_dummies(crypto_df)
X = features_enc
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,4.200000e+01,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,5.320000e+08,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,3.141593e+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,2.100000e+07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0.000000e+00,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Standardize data
crypto_scaler = StandardScaler()

crypto_scaled = crypto_scaler.fit_transform(X)
crypto_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Reducing Dimensions Using PCA

In [17]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [18]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index
)
pcs_df.head()

,PC 1,PC 2,PC 3
0,-0.344553,1.042835,-0.609817
2,-0.328001,1.042981,-0.610221
5,2.285489,1.740397,-0.701930
7,-0.144723,-1.246800,0.197034
8,-0.152161,-2.019850,0.438541


In [19]:
# Fetch the explained variance
pca.explained_variance_ratio_

array([0.02736784, 0.02094064, 0.02010753])

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [20]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(x='k', y='inertia').properties(
    title='The elbow method for determining best K value',
    width=400
)

alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [21]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
pcs_df["class"] = model.labels_ 

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = crypto_df.join(pcs_df)
clustered_df['CoinName'] = coins_name
clustered_df['Class'] = pcs_df["class"]

clustered_df.reset_index(inplace=True)
clustered_df.drop(columns=['class', 'index'], inplace=True)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.344553,1.042835,-0.609817,42 Coin,0
1,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.328001,1.042981,-0.610221,404Coin,0
2,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.285489,1.740397,-0.701930,EliteCoin,0
3,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.144723,-1.246800,0.197034,Bitcoin,1
4,Ethash,PoW,1.076842e+08,0.000000e+00,-0.152161,-2.019850,0.438541,Ethereum,1


### Visualizing Results

#### 2D-Scatter with Clusters

In [22]:
# Create a 2D-Scatter with the PCA data and the clusters
fig = alt.Chart(clustered_df).mark_circle(
    size=100
).encode(
    alt.X("PC 1", title="PC 1", scale=alt.Scale(zero=False)),
    alt.Y("PC 2", title="PC 2", scale=alt.Scale(zero=False)),
    color="Class",
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).properties(
    height=300,
    width=500,
    title="PC 1 vs PC 2 Color-encoded by Class").interactive()
fig

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [23]:
# Table with tradable cryptos
tradable_crypto_table = display(clustered_df.drop(columns=['PC 1', 'PC 2', 'PC 3']).head(10))

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,42 Coin,0
1,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,404Coin,0
2,X13,PoW/PoS,2.927942e+10,3.141593e+11,EliteCoin,0
3,SHA-256,PoW,1.792718e+07,2.100000e+07,Bitcoin,1
4,Ethash,PoW,1.076842e+08,0.000000e+00,Ethereum,1
5,Scrypt,PoW,6.303924e+07,8.400000e+07,Litecoin,1
6,X11,PoW/PoS,9.031294e+06,2.200000e+07,Dash,0
7,CryptoNight-V7,PoW,1.720114e+07,0.000000e+00,Monero,1
8,Ethash,PoW,1.133597e+08,2.100000e+08,Ethereum Classic,1
9,Equihash,PoW,7.383056e+06,2.100000e+07,ZCash,1


In [24]:
# Print the total number of tradable cryptocurrencies
total_trade_coins = len(clustered_df.CoinName)
print(f"The total number of tradable cryptocurrencies are {total_trade_coins}.")

The total number of tradable cryptocurrencies are 533.


#### Scatter Plot with Tradable Cryptocurrencies

In [25]:
# Scale data to create the scatter plot
scaler = MinMaxScaler()

clustered_columns = ["TotalCoinsMined", "TotalCoinSupply"]
x = clustered_df[clustered_columns].values
x_scaled = scaler.fit_transform(x)

In [26]:
# Create a dataframe with scaled data
tradable_coins_df = pd.DataFrame(
    data=x_scaled, columns=clustered_columns
)
tradable_coins_df.head()

,TotalCoinsMined,TotalCoinSupply
0,0.005942,4.200000e-11
1,0.007002,5.320000e-04
2,0.035342,3.141593e-01
3,0.005960,2.100000e-05
4,0.006050,0.000000e+00


In [27]:
# Drop the existing columns `TotalCoinsMined` and `TotalCoinSupply` from the `clustered_df` dataframe to avoid duplicates.
clustered_df.drop(columns=clustered_columns, inplace=True)
clustered_df.head()

,Algorithm,ProofType,PC 1,PC 2,PC 3,CoinName,Class
0,Scrypt,PoW/PoS,-0.344553,1.042835,-0.609817,42 Coin,0
1,Scrypt,PoW/PoS,-0.328001,1.042981,-0.610221,404Coin,0
2,X13,PoW/PoS,2.285489,1.740397,-0.701930,EliteCoin,0
3,SHA-256,PoW,-0.144723,-1.246800,0.197034,Bitcoin,1
4,Ethash,PoW,-0.152161,-2.019850,0.438541,Ethereum,1


In [28]:
# Concate the two dataframes `clustered_df` and `tradable_coins_df`
clustered_tradable_df = pd.concat([clustered_df, tradable_coins_df], axis='columns', join='inner')
clustered_tradable_df.head()

,Algorithm,ProofType,PC 1,PC 2,PC 3,CoinName,Class,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,-0.344553,1.042835,-0.609817,42 Coin,0,0.005942,4.200000e-11
1,Scrypt,PoW/PoS,-0.328001,1.042981,-0.610221,404Coin,0,0.007002,5.320000e-04
2,X13,PoW/PoS,2.285489,1.740397,-0.701930,EliteCoin,0,0.035342,3.141593e-01
3,SHA-256,PoW,-0.144723,-1.246800,0.197034,Bitcoin,1,0.005960,2.100000e-05
4,Ethash,PoW,-0.152161,-2.019850,0.438541,Ethereum,1,0.006050,0.000000e+00


In [29]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
fig2 = alt.Chart(clustered_tradable_df).mark_circle(
    size=100
).encode(
    alt.X("TotalCoinsMined", title="TotalCoinsMined", scale=alt.Scale(zero=False)),
    alt.Y("TotalCoinSupply", title="TotalCoinSupply", scale=alt.Scale(zero=False)),
    color="CoinName",
    tooltip=["CoinName", "TotalCoinsMined", "TotalCoinSupply"]
).properties(
    height=300,
    width=500,
    title="TotalCoinsMined versus TotalCoinSupply").interactive()
fig2

alt.Chart(...)